# New Bedford Code Violation Local Indicators of Spatial Association
#### Daniel Fay

In [40]:
# Import necessary packages
import pandas as pd
import geopandas as gpd
import numpy as np
import pysal as ps
import matplotlib.pyplot as plt
import seaborn
%matplotlib inline

In [47]:
# Import data
codeViolations = gpd.read_file('../Data/New_Bedford_CodeViolation_Shapefiles/code_violation_count.shp')
codeViolations.crs = {'init' :'epsg:4326'}


In [49]:
cvCount = codeViolations.Code_Viola
len(codeViolations)

1138

In [50]:
# Import and convert to spatial weigths
spatialWeights = ps.queen_from_shapefile('../Data/New_Bedford_CodeViolation_Shapefiles/code_violation_count.shp')

/Users/Danny1/anaconda/lib/python2.7/site-packages/pysal/weights/weights.py:190: UserWarning: Island ids: 188, 415, 462, 464, 967, 969, 1035, 1117
  warnings.warn("Island ids: %s" % ', '.join(str(island) for island in self.islands))


In [51]:
# Calculate spatial lag
spatialLag = ps.lag_spatial(spatialWeights, cvCount)

In [52]:
# Calculate Local Morans I
localMorans = ps.Moran_Local(cvCount, spatialWeights, permutations=999)

('WARNING: ', 188, ' is an island (no neighbors)')
('WARNING: ', 415, ' is an island (no neighbors)')
('WARNING: ', 462, ' is an island (no neighbors)')
('WARNING: ', 464, ' is an island (no neighbors)')
('WARNING: ', 967, ' is an island (no neighbors)')
('WARNING: ', 969, ' is an island (no neighbors)')
('WARNING: ', 1035, ' is an island (no neighbors)')
('WARNING: ', 1117, ' is an island (no neighbors)')


/Users/Danny1/anaconda/lib/python2.7/site-packages/pysal/esda/moran.py:835: RuntimeWarning: invalid value encountered in divide
  self.z_sim = (self.Is - self.EI_sim) / self.seI_sim


In [53]:
# Calculate Global Morans I
globalMorans = ps.Moran(cvCount, spatialWeights)

In [54]:
# Find significant values
sigs = cvCount[localMorans.p_sim <= .05]
insigs = cvCount[localMorans.p_sim > .05]

W_sigs = spatialLag[localMorans.p_sim <= .05]
W_insigs = spatialLag[localMorans.p_sim > .05]

In [ ]:
b,a = np.polyfit(cvCount, spatialLag, 1)

f, ax = plt.subplots(figsize=(10, 8))
plt.plot(sigs, W_sigs, '.', color='firebrick')
plt.plot(insigs, W_insigs, '.k', alpha=.4)
plt.text(s='$I = %.3f$' % globalMorans.I, x=100, y=100, fontsize=18)

 # dashed vert at mean of the last year's PCI
plt.vlines(cvCount.mean(), spatialLag.min(), spatialLag.max(), linestyle='--')
 # dashed horizontal at mean of lagged PCI
plt.hlines(spatialLag.mean(), cvCount.min(), cvCount.max(), linestyle='--')

# red line of best fit using global I as slope
plt.plot(cvCount, a + b*cvCount, 'r')
plt.title('Moran Scatterplot', fontsize=20)
plt.ylabel('Spatial Lag of Code Violations', fontsize=16)
plt.xlabel('Code Violations', fontsize=16)

In [55]:
sig = localMorans.p_sim < 0.05
insig = localMorans.p_sim >= 0.05
hot_hot = localMorans.q==1 * sig
cold_cold = localMorans.q==3 * sig
hot_cold= localMorans.q==2 * sig
cold_hot = localMorans.q==4 * sig

print ("Number of Hot-Hot spots:", hot_hot.sum())
print ("Number of Cold-Cold spots:", cold_cold.sum())
print ("Number of Hot-Cold spots:", hot_cold.sum())
print ("Number of Cold-Hot spots:", cold_hot.sum())

('Number of Hot-Hot spots:', 32)
('Number of Cold-Cold spots:', 139)
('Number of Hot-Cold spots:', 19)
('Number of Cold-Hot spots:', 16)


In [56]:
codeViolations['hot_hot'] = hot_hot.astype(int)
codeViolations['hot_cold'] = hot_cold.astype(int)
codeViolations['cold_cold'] = cold_cold.astype(int)
codeViolations['cold_hot'] = cold_hot.astype(int)
codeViolations['insig'] = insig.astype(int)
codeViolations['localMorans'] = localMorans.Is
codeViolations.head()

,Census_Tra,Code_Viola,geometry,hot_hot,hot_cold,cold_cold,cold_hot,insig,localMorans
0,250056501011001,6.722689,POLYGON ((-70.95696500049569 41.73801399964196...,0,0,0,0,1,0.065797
1,250056501011003,8.333333,"POLYGON ((-70.9530680005213 41.7323549996479, ...",0,0,0,0,1,-0.060103
2,250056501011006,28.571429,POLYGON ((-70.95271000000064 41.73432400033531...,0,0,0,0,1,-0.078469
3,250056501011007,12.500000,POLYGON ((-70.94453000055297 41.73790199969402...,0,0,0,0,1,0.032371
4,250056501011008,2.424242,"POLYGON ((-70.94141700052184 41.739900999809, ...",0,0,0,0,1,0.154124


In [57]:
codeViolations.to_file('../Data/New_Bedford_CodeViolation_Shapefiles/hot_cold_spots.shp')

In [58]:
codeViolations_LM = codeViolations[['Census_Tra', 'localMorans', 'geometry']]
codeViolations_LM.to_file('../Data/New_Bedford_CodeViolation_Shapefiles/local_morans.shp')